In [1]:
import numpy as np
import os
import pandas as pd

In [38]:
saveState = False
patNumbers = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
pd.set_option("display.precision", 3, "styler.format.decimal", ",")

# pointErrors = np.zeros(shape=(12, 5, 10)) # shape=(patients, reg_stages, distances)
medianErrorsStages = np.zeros(shape=(12, 5)) # shape=(patients, reg_stages)
# preInitialMedians = np.zeros(shape=(12, 2, 10))
# lastIterMediands = np.zeros(shape=(12, 3, 10))
# initialLastmedianErrors = np.zeros(shape=(12, 5))
runTime = np.zeros(shape=(12, 3))
lastIterMetrics = np.zeros(shape=(12, 3))

for idx, num in enumerate(patNumbers):
    errorPath = f"input_files\\pacient_{num}\\registration\\"

    preFile = np.load(f"{errorPath}gradient\\pacient{num}PreErrors.npz")
    initialFile = np.load(f"{errorPath}gradient\\pacient{num}InitialErrors.npz")
    gradFile = np.load(f"{errorPath}gradient\\pacient{num}RegErrors.npz")
    gradLineFile = np.load(f"{errorPath}gradientline\\pacient{num}RegErrors.npz")
    gradLbfFile = np.load(f"{errorPath}gradientlbf\\pacient{num}RegErrors.npz")
    
    gradRegFile = np.load(f"{errorPath}gradient\\pacient{num}RegParams.npz")
    gradLineRegFile = np.load(f"{errorPath}gradientline\\pacient{num}RegParams.npz")
    gradLbfRegFile = np.load(f"{errorPath}gradientlbf\\pacient{num}RegParams.npz")
    # pointErrors[idx, 0, :] = preFile["preErrors"]
    # pointErrors[idx, 1, :] = initialFile["initialErrors"]
    # pointErrors[idx, 2, :] = gradFile["regErrors"][-1, :]
    # pointErrors[idx, 3, :] = gradLineFile["regErrors"][-1, :]
    # pointErrors[idx, 4, :] = gradLbfFile["regErrors"][-1, :]

    # median errors per patient and registration stage
    medianErrorsStages[idx, 0] = preFile["preMedians"] # before initialization
    medianErrorsStages[idx, 1] = initialFile["initialMedians"] # after initialization
    medianErrorsStages[idx, 2] = gradFile["regMedians"][-1] # gradient 
    medianErrorsStages[idx, 3] = gradLineFile["regMedians"][-1] # gradient line 
    medianErrorsStages[idx, 4] = gradLbfFile["regMedians"][-1] # lbfgs
    
    lastIterMetrics[idx, 0] = gradRegFile["metricValues"][-1]
    lastIterMetrics[idx, 1] = gradLineRegFile["metricValues"][-1]
    lastIterMetrics[idx, 2] = gradLbfRegFile["metricValues"][-1]
    
    runTime[idx, 0] = gradRegFile["execTime"]
    runTime[idx, 1] = gradLineRegFile["execTime"]
    runTime[idx, 2] = gradLbfRegFile["execTime"]
    
    del(preFile, initialFile, gradFile, gradLineFile, gradLbfFile, gradRegFile,
        gradLineRegFile, gradLbfRegFile)
    

regStages = ["Před inicializací", "Po inicializaci", "Gradientní\nsestup", 
             "Gradientní sestup\nse zl. řezem", "BFGS s\nomezenou pamětí"]
regCols = ["pre", "initial", "grad", 
             "grad_line", "grad_lbf"]
regLabels = ["Před inicializací", "Po inicializaci", "Gradientní sestup", 
             "Gradientní sestup se zl. řezem", "BFGS s omezenou pamětí"]

In [54]:
initialLastmedianErrorsDf = pd.DataFrame(medianErrorsStages, columns=pd.Index(regLabels),
                                       index=pd.Index(range(1, medianErrorsStages.shape[0] + 1), name="Číslo pacienta"))
print(initialLastmedianErrorsDf.T.style.to_latex())
initialLastmedianErrorsDf.T

\begin{tabular}{lrrrrrrrrrrrr}
Číslo pacienta & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 \\
Před inicializací & 95,204770 & 99,369106 & 59,651445 & 77,284026 & 84,555336 & 125,291004 & 51,334492 & 25,338598 & 117,979494 & 30,762199 & 44,756299 & 115,841176 \\
Po inicializaci & 102,126586 & 79,781340 & 23,645868 & 83,098711 & 20,322598 & 60,211795 & 33,765366 & 14,480076 & 62,515038 & 26,047366 & 32,823615 & 42,758684 \\
Gradientní sestup & 11,128011 & 22,416605 & 6,912589 & 13,471545 & 11,880615 & 11,366609 & 5,633837 & 4,631353 & 4,237906 & 19,306719 & 22,091272 & 4,638175 \\
Gradientní sestup se zl. řezem & 12,286658 & 22,286418 & 5,379593 & 12,334207 & 11,369002 & 10,927066 & 6,153505 & 6,041925 & 3,674213 & 16,737000 & 20,876263 & 4,129624 \\
BFGS s omezenou pamětí & 12,918571 & 22,440662 & 5,396685 & 11,924426 & 10,999531 & 10,251254 & 6,370661 & 6,180540 & 3,577489 & 15,500227 & 21,230581 & 4,541347 \\
\end{tabular}


Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Před inicializací,95.205,99.369,59.651,77.284,84.555,125.291,51.334,25.339,117.979,30.762,44.756,115.841
Po inicializaci,102.127,79.781,23.646,83.099,20.323,60.212,33.765,14.480,62.515,26.047,32.824,42.759
Gradientní sestup,11.128,22.417,6.913,13.472,11.881,11.367,5.634,4.631,4.238,19.307,22.091,4.638
Gradientní sestup se zl. řezem,12.287,22.286,5.380,12.334,11.369,10.927,6.154,6.042,3.674,16.737,20.876,4.130
BFGS s omezenou pamětí,12.919,22.441,5.397,11.924,11.000,10.251,6.371,6.181,3.577,15.500,21.231,4.541


In [53]:
initialLastMetricsDf = pd.DataFrame(lastIterMetrics, columns=pd.Index(regLabels[2:]),
                                    index=pd.Index(range(0, lastIterMetrics.shape[0]), name="Číslo pacienta"))
print(initialLastMetricsDf.T.style.to_latex())
initialLastMetricsDf.T

\begin{tabular}{lrrrrrrrrrrrr}
Číslo pacienta & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 \\
Gradientní sestup & -0,638613 & -0,453557 & -0,654914 & -0,593322 & -0,627951 & -0,645471 & -0,670310 & -0,623142 & -0,690821 & -0,626787 & -0,629990 & -0,661599 \\
Gradientní sestup se zl. řezem & -0,644408 & -0,457392 & -0,663807 & -0,594194 & -0,629535 & -0,647408 & -0,675961 & -0,632323 & -0,703719 & -0,634804 & -0,633871 & -0,670891 \\
BFGS s omezenou pamětí & -0,644307 & -0,456739 & -0,657225 & -0,596790 & -0,629365 & -0,645745 & -0,675967 & -0,624496 & -0,703737 & -0,634184 & -0,633951 & -0,670570 \\
\end{tabular}


Číslo pacienta,0,1,2,3,4,5,6,7,8,9,10,11
Gradientní sestup,-0.639,-0.454,-0.655,-0.593,-0.628,-0.645,-0.670,-0.623,-0.691,-0.627,-0.630,-0.662
Gradientní sestup se zl. řezem,-0.644,-0.457,-0.664,-0.594,-0.630,-0.647,-0.676,-0.632,-0.704,-0.635,-0.634,-0.671
BFGS s omezenou pamětí,-0.644,-0.457,-0.657,-0.597,-0.629,-0.646,-0.676,-0.624,-0.704,-0.634,-0.634,-0.671


In [51]:
runTimeDf = pd.DataFrame(runTime, columns=pd.Index(regLabels[2:]), 
                         index=pd.Index(range(1, runTime.shape[0] + 1), name="Číslo pacienta"))
print(runTimeDf.T.style.to_latex())
runTimeDf.T

\begin{tabular}{lrrrrrrrrrrrr}
Číslo pacienta & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 \\
Gradientní sestup & 1,567274 & 1,642866 & 1,786077 & 1,800412 & 2,177751 & 1,625393 & 1,387234 & 1,520851 & 1,467951 & 2,657691 & 2,452025 & 1,557656 \\
Gradientní sestup se zl. řezem & 4,869071 & 4,075866 & 5,323518 & 8,581331 & 5,901885 & 5,188971 & 4,107326 & 4,381790 & 4,544885 & 8,750057 & 8,418606 & 6,094152 \\
BFGS s omezenou pamětí & 6,096101 & 4,126926 & 7,020240 & 5,620288 & 6,865702 & 6,016919 & 6,331690 & 5,360708 & 5,723022 & 7,290385 & 9,208305 & 4,714112 \\
\end{tabular}


Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Gradientní sestup,1.567,1.643,1.786,1.800,2.178,1.625,1.387,1.521,1.468,2.658,2.452,1.558
Gradientní sestup se zl. řezem,4.869,4.076,5.324,8.581,5.902,5.189,4.107,4.382,4.545,8.750,8.419,6.094
BFGS s omezenou pamětí,6.096,4.127,7.020,5.620,6.866,6.017,6.332,5.361,5.723,7.290,9.208,4.714


In [55]:
with pd.ExcelWriter("graficke_vystupy\\vystupy_registrace.xlsx") as excel_writer:
    initialLastmedianErrorsDf.to_excel(excel_writer, sheet_name="median_mezibodove_vzdalenosti",
                                       float_format="%.3f",
                                       columns=initialLastmedianErrorsDf.columns)
    
    runTimeDf.to_excel(excel_writer, sheet_name="vypocetni_doba",
                       float_format="%.3f", 
                       columns=runTimeDf.columns)
    
    initialLastMetricsDf.to_excel(excel_writer, sheet_name="vzajemna_informace",
                                  float_format="%.3f",
                                  columns=initialLastMetricsDf.columns)